# Training a Convolutional Neural Network

In this notebook, we will create a CNN model using PyTorch and then train it on the CIFAR10 dataset. 


## Package Installations
**NOTE**: Everytime you start the GPU, run this before your code. 

In [1]:
!pip install ipywidgets
#!pip list


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python3.8 -m pip install --upgrade pip


In [2]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time # for measuring time for testing, remove for students

def test(model, test_loader, criterion, device):
    print("Testing Model on Whole Testing Dataset")
    model.eval()
    running_loss=0
    running_corrects=0
    
    for inputs, labels in test_loader:
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs=model(inputs)
        loss=criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    total_loss = running_loss / len(test_loader.dataset)
    total_acc = running_corrects/ len(test_loader.dataset)
    print(f"Testing Accuracy: {100*total_acc}, Testing Loss: {total_loss}")
    
def train(model, train_loader, validation_loader, criterion, optimizer, device):
    epochs=2
    best_loss=1e6
    image_dataset={'train':train_loader, 'valid':validation_loader}
    loss_counter=0
    
    for epoch in range(epochs):
        for phase in ['train', 'valid']:
            print(f"Epoch {epoch}, Phase {phase}")
            if phase=='train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            running_samples=0

            for step, (inputs, labels) in enumerate(image_dataset[phase]):
                inputs=inputs.to(device)
                labels=labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase=='train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
                running_samples+=len(inputs)
                if running_samples % 2000  == 0:
                    accuracy = running_corrects/running_samples
                    print("Images [{}/{} ({:.0f}%)] Loss: {:.2f} Accuracy: {}/{} ({:.2f}%) Time: {}".format(
                            running_samples,
                            len(image_dataset[phase].dataset),
                            100.0 * (running_samples / len(image_dataset[phase].dataset)),
                            loss.item(),
                            running_corrects,
                            running_samples,
                            100.0*accuracy,
                            time.asctime() # for measuring time for testing, remove for students and in the formatting
                        )
                    )
                
                #NOTE: Comment lines below to train and test on whole dataset
                if running_samples>(0.2*len(image_dataset[phase].dataset)):
                    break

            epoch_loss = running_loss / running_samples
            epoch_acc = running_corrects / running_samples
            
            if phase=='valid':
                if epoch_loss<best_loss:
                    best_loss=epoch_loss
                else:
                    loss_counter+=1

        if loss_counter==1:
            break
    return model

def create_model():
    model = models.resnet18(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False   

    num_features=model.fc.in_features
    model.fc = nn.Sequential(
                   nn.Linear(num_features, 10))
    return model

batch_size=10

if torch.backends.mps.is_available(): # for Apple Silicon users
    device = torch.device("mps")
elif torch.cuda.is_available(): 
    torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(f"Running on Device {device}")

training_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

testing_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
        download=True, transform=training_transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
        shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
        download=True, transform=testing_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
        shuffle=False)

model=create_model()
model=model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

train(model, trainloader, testloader, criterion, optimizer, device)

test(model, testloader, criterion, device)

Running on Device mps
Files already downloaded and verified
Files already downloaded and verified
Epoch 0, Phase train
Images [2000/50000 (4%)] Loss: 1.10 Accuracy: 873/2000 (43.65%) Time: Sat Feb  4 10:51:36 2023
Images [4000/50000 (8%)] Loss: 1.16 Accuracy: 2129/4000 (53.23%) Time: Sat Feb  4 10:51:54 2023
Images [6000/50000 (12%)] Loss: 1.24 Accuracy: 3453/6000 (57.55%) Time: Sat Feb  4 10:52:12 2023
Images [8000/50000 (16%)] Loss: 0.96 Accuracy: 4861/8000 (60.76%) Time: Sat Feb  4 10:52:31 2023
Images [10000/50000 (20%)] Loss: 0.60 Accuracy: 6288/10000 (62.88%) Time: Sat Feb  4 10:52:50 2023
Epoch 0, Phase valid
Images [2000/10000 (20%)] Loss: 0.49 Accuracy: 1443/2000 (72.15%) Time: Sat Feb  4 10:53:01 2023
Epoch 1, Phase train
Images [2000/50000 (4%)] Loss: 0.78 Accuracy: 1363/2000 (68.15%) Time: Sat Feb  4 10:53:19 2023
Images [4000/50000 (8%)] Loss: 0.85 Accuracy: 2759/4000 (68.97%) Time: Sat Feb  4 10:53:37 2023
Images [6000/50000 (12%)] Loss: 0.30 Accuracy: 4167/6000 (69.45%) 

## Try It Out!
- Play around with the number of layers and filters in your model. How does the accuracy change? How long does it take to train the model?
- Try to train your model with some other types of convolutional layers like depthwise separable convolutions
- Can you create the same network in TensorFlow as well?

# Fine tuning CNN

Here we will be fine tuning Resnet 18 on CIFAR10 dataset.

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time # for measuring time for testing, remove for students

def test(model, test_loader, criterion, device):
    print("Testing Model on Whole Testing Dataset")
    model.eval()
    running_loss=0
    running_corrects=0
    
    for inputs, labels in test_loader:
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs=model(inputs)
        loss=criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    total_loss = running_loss / len(test_loader.dataset)
    total_acc = running_corrects/ len(test_loader.dataset)
    print(f"Testing Accuracy: {100*total_acc}, Testing Loss: {total_loss}")
    
def train(model, train_loader, validation_loader, criterion, optimizer, device):
    epochs=2
    best_loss=1e6
    image_dataset={'train':train_loader, 'valid':validation_loader}
    loss_counter=0
    
    for epoch in range(epochs):
        for phase in ['train', 'valid']:
            print(f"Epoch {epoch}, Phase {phase}")
            if phase=='train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            running_samples=0

            for step, (inputs, labels) in enumerate(image_dataset[phase]):
                inputs=inputs.to(device)
                labels=labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase=='train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
                running_samples+=len(inputs)
                if running_samples % 2000  == 0:
                    accuracy = running_corrects/running_samples
                    print("Images [{}/{} ({:.0f}%)] Loss: {:.2f} Accuracy: {}/{} ({:.2f}%)".format(
                            running_samples,
                            len(image_dataset[phase].dataset),
                            100.0 * (running_samples / len(image_dataset[phase].dataset)),
                            loss.item(),
                            running_corrects,
                            running_samples,
                            100.0*accuracy,
                        )
                    )
                
                #NOTE: Comment lines below to train and test on whole dataset
                if running_samples>(0.2*len(image_dataset[phase].dataset)):
                    break

            epoch_loss = running_loss / running_samples
            epoch_acc = running_corrects / running_samples
            
            if phase=='valid':
                if epoch_loss<best_loss:
                    best_loss=epoch_loss
                else:
                    loss_counter+=1

        if loss_counter==1:
            break
    return model

def create_model():
    model = models.resnet18(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False   

    num_features=model.fc.in_features
    model.fc = nn.Sequential(
                   nn.Linear(num_features, 10))
    return model

batch_size=10

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    torch.device("cuda:0")
else:
    device = torch.device("cpu")
    
print(f"Running on Device {device}")

training_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

testing_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
        download=True, transform=training_transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
        shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
        download=True, transform=testing_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
        shuffle=False)

model=create_model()
model=model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

train(model, trainloader, testloader, criterion, optimizer, device)

test(model, testloader, criterion, device)

Running on Device mps
Files already downloaded and verified
Files already downloaded and verified
Epoch 0, Phase train
Images [2000/50000 (4%)] Loss: 1.27 Accuracy: 857/2000 (42.85%)
Images [4000/50000 (8%)] Loss: 0.68 Accuracy: 2110/4000 (52.75%)
Images [6000/50000 (12%)] Loss: 0.89 Accuracy: 3499/6000 (58.32%)
Images [8000/50000 (16%)] Loss: 0.79 Accuracy: 4869/8000 (60.86%)
Images [10000/50000 (20%)] Loss: 1.30 Accuracy: 6251/10000 (62.51%)
Epoch 0, Phase valid
Images [2000/10000 (20%)] Loss: 0.54 Accuracy: 1463/2000 (73.15%)
Epoch 1, Phase train
Images [2000/50000 (4%)] Loss: 0.48 Accuracy: 1391/2000 (69.55%)
Images [4000/50000 (8%)] Loss: 0.69 Accuracy: 2818/4000 (70.45%)
Images [6000/50000 (12%)] Loss: 1.31 Accuracy: 4245/6000 (70.75%)
Images [8000/50000 (16%)] Loss: 0.81 Accuracy: 5658/8000 (70.73%)
Images [10000/50000 (20%)] Loss: 0.53 Accuracy: 7104/10000 (71.04%)
Epoch 1, Phase valid
Images [2000/10000 (20%)] Loss: 0.64 Accuracy: 1564/2000 (78.20%)
Testing Model on Whole Test